# Code to Generate Biochar Atomistic Models

The code is applicable for molar ratios of O/C ≤ 0.33 and H/C ≤ 0.89 and elemental composition, including C, H, N, O, F, P, and S.

<img src="biochar.png" width="450" height="450">


## Open Pymol 

In [1]:
# Import pymol
from ipymol import viewer as pymol

In [2]:
# Initiate PyMol RPC module
import glob,os,sys,subprocess,random, math
from pymol_rpc import pymol_jupyter_builder
import numpy as np
from scipy.optimize import minimize
from sympy import symbols, Eq, solve

builder = pymol_jupyter_builder()
builder.start()

2025-01-07 18:25:26.770 python[39364:664265] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-07 18:25:26.770 python[39364:664265] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [3]:
# Import cmd 
import xmlrpc.client as xmlrpclib
cmd = xmlrpclib.ServerProxy('http://localhost:9123')
cmd.reinitialize()

## Input: Biochar data

In [4]:
Biochar_Temp=float(input("Enter the temperature of pyrolysis in °C: "))

Enter the temperature of pyrolysis in °C:  500


### Ultimate analysis

In [5]:
Carbon = float(input("Enter the value for Carbon: "))
Hydrogen = float(input("Enter the value for Hydrogen: "))
Oxygen = float(input("Enter the value for Oxygen: "))
Nitrogen = float(input("Enter the value for Nitrogen: "))
Fluorine = float(input("Enter the value for Fluorine: "))
Phosphorus = float(input("Enter the value for Phosphorus: "))
Sulfur = float(input("Enter the value for Sulfur: "))
H_C = float(input("Enter the value for H/C molar ratio: "))
O_C = float(input("Enter the value for O/C molar ratio: "))
BridgheadCarbon=float(input("Enter the value for BridgheadCarbon: "))

Enter the value for Carbon:  0.7882
Enter the value for Hydrogen:  0.0470
Enter the value for Oxygen:  0.1538
Enter the value for Nitrogen:  0.0033
Enter the value for Fluorine:  0.0245
Enter the value for Phosphorus:  0
Enter the value for Sulfur:  0
Enter the value for H/C molar ratio:  0.71
Enter the value for O/C molar ratio:  0.15
Enter the value for BridgheadCarbon:  0.29


In [6]:
# 1 H2SO4
# 2 H3PO4
# 3 CH2O2
# 4 CH3COOH
Acid=float(input("Enter the acid used for the pre-treatment: "))

Enter the acid used for the pre-treatment:  3


In [7]:
# Hydroxyl groups, qualitative data from XPS O1s O–C bonds

if Acid == 1:
    v = 0.6256*0.8  # Include the contribution of O–S bonds
elif Acid == 2:
    v = 0.7468*0.15 # Includes the contribution of O–P bonds; for each P atom there are 4 O–P bonds 
elif Acid == 3:
    v = 0.5273
else:
    v = 0.7237*0.7

In [8]:
# Sulfur groups, qualitative data from XPS S 2p1/2 and 2p3/2
Thiophene=0.4964          # C–S–C (Carbon from 5 member ring)
Thioether=0.248           # C–S–C (No aromatic carbon)
Sulfonyl=0.1704/2         # SO2
Sulfo=0.1704/2            # SO3H
Sulfinyl=0.0851           # S=O

In [9]:
# Phosphorus groups, qualitative data from XPS P 2p1/2 and 2p3/2
Phosphate=0.6668         # C–O–PO3
Phosphite=0.3332         # C–PO3

### Multi-CP 13C NMR quantitaive data

In [10]:
Aromatic = float(input("Enter the value for Caro: "))
Carbonyl = float(input("Enter the value for Carbonyl: "))
Ester = float(input("Enter the value for Ester: "))          # Carboxyl/Lactone/Ester
Ether = float(input("Enter the value for Ether: "))
Aliphatic = float(input("Enter the value for Aliphatic: "))
Defect = float(input("Enter the value for Defect: "))        # Non-hexagonal rings

Enter the value for Caro:  0.68
Enter the value for Carbonyl:  0
Enter the value for Ester:  0.01
Enter the value for Ether:  0.11
Enter the value for Aliphatic:  0.13
Enter the value for Defect:  0.07


### Aromatic cluster size distribution, obtained from BPCA.ipynb

In [11]:
system_size = float(input("System size selected on BPCA.ipynb: "))

System size selected on BPCA.ipynb:  10000


In [12]:
%store -r result
values=np.round(result[0]).astype(int)
print(values)

[ 0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  2 66  0  0  0  8  0
 17  0  0  0  0  0  0  0  0]


## Step1: Include the aromatic clusters in PyMOL and create the grid

#### Note: All the molecules included in the BPCA_PAH should be in this folder as mol2 file 

In [13]:
cmd.load("fluorene.mol2", "fluorene")
cmd.load("phenalene.mol2", "phenalene")
cmd.load("phenanthrene.mol2", "phenanthrene")
cmd.load("anthracene.mol2", "anthracene")
cmd.load("tetracene.mol2", "tetracene")
cmd.load("pentacene.mol2", "pentacene")
cmd.load("pyrene.mol2", "pyrene")
cmd.load("chrysene.mol2", "chrysene")
cmd.load("benzo_a_fluorene.mol2", "benzo_a_fluorene")
cmd.load("benzo_b_fluoranthene.mol2", "benzo_b_fluoranthene")
cmd.load("benzo_b_fluorene.mol2", "benzo_b_fluorene")
cmd.load("coronene.mol2", "coronene")
cmd.load("perylene.mol2", "perylene")
cmd.load("benzo_a_pyrene.mol2", "benzo_a_pyrene")
cmd.load("benzo_g_h_i_perylene.mol2", "benzo_g_h_i_perylene")
cmd.load("circumpyrene.mol2", "circumpyrene")
cmd.load("circumcoronene.mol2", "circumcoronene")
cmd.load("circumovalene.mol2", "circumovalene")
cmd.load("pentatriacotaene.mol2", "pentatriacotaene")
cmd.load("circumcircumpyrene.mol2", "circumcircumpyrene")
cmd.load("c84.mol2", "c84")
cmd.load("N5.mol2", "m5")
cmd.load("N8.mol2", "N8")
cmd.load("N9.mol2", "N9")
cmd.load("N10.mol2", "N10")
cmd.load("N11.mol2", "N11")
cmd.load("N12.mol2", "N12")
cmd.load("N13.mol2", "N13")
cmd.load("N14.mol2", "N14")
cmd.load("N15.mol2", "N15")
cmd.load("N16.mol2", "N16")
cmd.load("N17.mol2", "N17")
cmd.load("N18.mol2", "N18")

1

In [14]:
cmd.set_color("pale_yellow_green", [0.7, 0.9, 0.5])
cmd.alias("colour", "color cyan, (name c*); color red, (name o*); color white, (name h*); color blue, (name n*); color yellow, (name s*); color brown, (name p*); color pale_yellow_green, (name f*)")

In [15]:
cmd.do('colour')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')
cmd.bg_color("white")
cmd.set("ray_shadows", "off")

In [16]:
def generate_coordinates(num_coordinates):
    coordinates = []
    
    xmin, xmax = 0, 150
    ymin, ymax = -150, 0
    zmin, zmax = 0, 150

    num_per_dim = int(math.ceil(num_coordinates ** (1/3)))
    
    x_spacing = (xmax - xmin) / (num_per_dim - 1)
    y_spacing = (ymax - ymin) / (num_per_dim - 1)
    z_spacing = (zmax - zmin) / (num_per_dim - 1)
    
    count = 0
    for x in range(num_per_dim):
        for y in range(num_per_dim):
            for z in range(num_per_dim):
                if count >= num_coordinates:
                    break
                x_coord = xmin + x * x_spacing
                y_coord = ymin + y * y_spacing
                z_coord = zmin + z * z_spacing
                rotation_angles = [random.uniform(0, 360) for _ in range(3)]
                coordinates.append((x_coord, y_coord, z_coord, rotation_angles))
                count += 1
            if count >= num_coordinates:
                break
        if count >= num_coordinates:
            break

    return coordinates

coordinates = generate_coordinates(round(system_size*0.03))

In [17]:
object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH types:", num_objects)

PAH types: 33


#### Insert all the PAH in PyMol according to the BPCA distribution

In [18]:
def copy_translate_delete(object_name, num_copies, coordinates, start_index):
    for i in range(1, num_copies + 1):
        cmd.copy(f"{object_name}{i}", object_name)
        cmd.translate(coordinates[start_index + i - 1], f"{object_name}{i}")
    cmd.delete(object_name)

for idx, object_name in enumerate(object_list):
    copy_translate_delete(object_name, values[idx], coordinates, num_objects)
    num_objects += values[idx]

cmd.reset()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH clusters:", num_objects)

PAH clusters: 102


### Check the number of aromatic carbons

In [19]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_aromatic = atom_counts["C"]
h_aromatic = atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5727
Element H: 2070
Element N: 0
Element O: 0
Element S: 0
Element P: 0
Element F: 0


## Step 2: Include cluster with non-hexagonal rings

#### Include a frequency for the Carbon Defects, to have the complete distribution of fragments.

In [20]:
# 1 Yes, 0 No
holes = float(input("Do you want to include holes within the structures?: "))  

Do you want to include holes within the structures?:  1


In [21]:
total_atoms=c_aromatic+h_aromatic
h_c_aro=h_aromatic/c_aromatic
def calculate_factors(Biochar_Temp, total_atoms):
    if holes == 0:
        if Acid == 1:
            w = -1 + H_C
            y = 2.3 * (h_c_aro/H_C)
            f = round(system_size*0.002)
        elif Acid == 2:
            w = 1.5 + H_C
            y = 2.5 * (h_c_aro/H_C)
            f = round(system_size*0.002)
        elif Acid == 3:
            w = 1 + H_C
            y = 2.23 * (h_c_aro/H_C)
            f = round(system_size*0.003)
        elif Acid == 4:
            w = -0.3
            y = 3.4 * (h_c_aro/H_C)
            f = round(system_size*0.003)
    else:  
        if Acid == 1:
            w = -1 + H_C
            y = 2.5 * (h_c_aro/H_C)
            f = round(system_size*0.002)
        elif Acid == 2:
            w =  1.5 * H_C
            y = 2.7 * (h_c_aro/H_C)
            f = round(system_size*0.002)
        elif Acid == 3:
            w = 1 + H_C
            y = 2.3 * (h_c_aro/H_C)
            f = round(system_size*0.002)
        elif Acid == 4:
            w = -0.26
            y = 3.5 * (h_c_aro/H_C)
            f = round(system_size*0.002)
              
    return w, y, f

w, y, f = calculate_factors(Biochar_Temp, total_atoms)
print("Adjusted factors - w:", w, "y:", y, "f:", f)

Adjusted factors - w: 1.71 y: 1.1708807059222808 f: 20


In [22]:
def_list = [61, 74, 93, 42, 45, 162, 67, 183, 192, 64, 72, 93]
def_list_H = [23, 96, 126, 22, 32, 177, 83, 178, 175, 74, 28, 136]
TotalC_Aromatic = c_aromatic
TotalH_Aromatic = h_aromatic

value =[]

def constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic):
    
    # Total number of defective carbons
    value = [(def_list[i] * vector[i]) / (TotalC_Aromatic + (def_list[i] * vector[i])) for i in range(len(def_list))]
    constraint_value = sum(value)
     
    # Total H/C
    value_H_d = [def_list_H[i] * vector[i] for i in range(len(def_list))]
    value_C_d =  [def_list[i] * vector[i] for i in range(len(def_list))]      
    value_HC = (sum(value_H_d) + TotalH_Aromatic) / (TotalC_Aromatic + sum(value_C_d))
    
    # Total % of carbons
    value_C = (sum(value_C_d)  + TotalC_Aromatic) / (TotalC_Aromatic + TotalH_Aromatic + sum(value_H_d) + sum(value_C_d))
    constraint_value_C = value_C

    # Minimize errors
    error_HC = np.abs(value_HC - (H_C+w))/(H_C+w) # We need more hydrogen to create cross-links    
    error_Def = np.abs(constraint_value - Defect*y)/(Defect*y) # We need more carbon defect to create holes
    
    Total_error = error_Def + error_HC

    return Total_error*100

# Initial guess for the optimization
initial_vector = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

bounds = [(0, f) for i in range(len(def_list))]

# Optimize the objective function (set to zero) subject to constraints
result = minimize(lambda x: 0, initial_vector, method='slsqp', constraints={'type': 'eq', 'fun': constraint, 'args': (def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic)}, bounds=bounds, tol=1e-5)

# Extract the optimized vector
vector = result.x.round().astype(int)

# Print the results
print("Value:", constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic))
print("Optimized Vector:", vector)

Value: 91.98354004031687
Optimized Vector: [0 1 2 0 0 0 0 0 0 0 0 3]


In [23]:
# New vector with defective structures included
values = np.append(values, vector).astype(int)
print(values)

[ 0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  2 66  0  0  0  8  0
 17  0  0  0  0  0  0  0  0  0  1  2  0  0  0  0  0  0  0  0  3]


In [24]:
# Upload defects
cmd.load("defect1.mol2", "defect0")
cmd.load("defect2.mol2", "defect2")
cmd.load("defect3.mol2", "defect3")
cmd.load("defect4.mol2", "defect4")
cmd.load("defect5.mol2", "defect5")
cmd.load("defect6.mol2", "defect6")
cmd.load("defect7.mol2", "defect7")
cmd.load("defect8.mol2", "defect8")
cmd.load("defect9.mol2", "defect9")
cmd.load("defect10.mol2", "defect10")
cmd.load("defect11.mol2", "defect11")
cmd.load("defect12.mol2", "defect12")

1

In [25]:
# List of defect object names and their corresponding indices in vector `values`
defect_object_names = [
    ("defect0", 33),
    ("defect3", 35),
    ("defect2", 34),
    ("defect4", 36),
    ("defect5", 37),
    ("defect6", 38),
    ("defect7", 39),
    ("defect8", 40),
    ("defect9", 41),
    ("defect10", 42),
    ("defect11", 43),
    ("defect12", 44)
]

defect_offset = num_objects

for object_name, value_index in defect_object_names:
    copy_translate_delete(object_name, values[value_index], coordinates, defect_offset)
    defect_offset += values[value_index]

cmd.reset()
object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)
cmd.reset()

Number of objects: 108


1

In [26]:
cmd.do('colour')
cmd.do('comp')
cmd.do('atomdata')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

### Organize the structures without overlapping them; this will allow you to visualize better the changes made

In [27]:
min_angle = 1
max_angle = 359.0

num_objects = len(object_list)

positions = []
used_positions = set()
angles = []
xmin, xmax = 0, round(system_size*0.015)
ymin, ymax = -round(system_size*0.015), 0
zmin, zmax = 0, round(system_size*0.015)

# Minimum distance between any two positions to avoid overlap
min_distance = round(system_size*0.005)
grid_divisions = int(math.ceil(num_objects ** (1 / 3)))

grid_size_x = min_distance
grid_size_y = min_distance
grid_size_z = min_distance

# Generate all possible grid positions
all_positions = [
    (xmin + i * grid_size_x, ymin + j * grid_size_y, zmin + k * grid_size_z)
    for i in range(grid_divisions)
    for j in range(grid_divisions)
    for k in range(grid_divisions)
]

random.shuffle(all_positions)
assert len(all_positions) >= num_objects, "Not enough positions for all objects."

# Assign positions to objects
for idx, name in enumerate(object_list):
    x, y, z = all_positions[idx]
    positions.append((x, y, z))

    rotation_angles = [random.uniform(min_angle, max_angle) for _ in range(3)]
    angles.append((rotation_angles, name))

    for axis, angle in zip(['x', 'y', 'z'], rotation_angles):
        rotation_command = "rotate {0}, {1}, {2}".format(axis, angle, name)
        cmd.do(rotation_command)

    # Translate the object to the new position
    cmd.translate((x, y, z), name)

cmd.refresh()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)

Number of objects: 108


In [28]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6266
Element H: 2820
Element N: 0
Element O: 0
Element S: 0
Element P: 0
Element F: 0


In [29]:
c_defect=c_total-c_aromatic
print("Defect carbon:", c_defect)

Defect carbon: 539


## Step 3: Include functional groups

### Add additional groups as needed

In [30]:
# To reduce the error of not having enough information for oxygen, it is necessary to sacrifice the precision of functional groups 
# that contain oxygen, such as ether, carbonyl, ester, and carboxylic groups.
if holes == 1:
    if Acid ==3:
        z = -0.07     # A factor to fix oxygen based on ether, carbonyl, and ester groups
        u = 0         # How much of the aliphatic is methyl
        b = 0.18      # How much of the aliphatic is CH2CH3
        p = 0.82      # How much of the aliphatic is (CH2)-(CH2)-CH3
        k = 0         # How much of the aliphatic is (CH)3-CH2
        h = -0.07     # A factor to fix oxygen based on aliphatic groups
        q = 0.02      # A factor to fix oxygen based on defects
        g = 1
    
    elif Acid ==4:
        z = 0.04
        u = 0
        b = 0
        p = 0.4
        k = 0.6
        h = 0.1
        q = -0.05
        g = 1

    elif Acid ==1:
        z = -0.05
        u = 0
        b = 0.1
        p = 0.25
        k = 0.65
        h = 0.07
        q = -0.005
        g = 1
    
    else:
        z = 0
        u = 0.7
        b = 0.2
        p = 0.1
        k = 0
        h = 0.0
        q = 0.05
        g = 0.98

else:
    if Acid ==3:
        z = -0.07     # A factor to fix oxygen based on ether, carbonyl, and ester groups
        u = 0         # How much of the aliphatic is methyl
        b = 0.18      # How much of the aliphatic is CH2CH3
        p = 0.82      # How much of the aliphatic is (CH2)-(CH2)-CH3
        k = 0         # How much of the aliphatic is (CH)3-CH2
        h = -0.07     # A factor to fix oxygen based on aliphatic groups
        q = 0.0       # A factor to fix oxygen based on defects
        g = 1
    
    elif Acid ==4:
        z = 0.04
        u = 0
        b = 0
        p = 0.4
        k = 0.6
        h = 0.1
        q = 0
        g = 1

    elif Acid ==1:
        z = -0.05
        u = 0
        b = 0.1
        p = 0.25
        k = 0.65
        h = 0.07
        q = 0
        g = 1
    
    else:
        z = 0
        u = 0.7
        b = 0.2
        p = 0.1
        k = 0
        h = 0.0
        q = 0
        g = 0.98

### Fix carbon, nitrogen, oxygen, and hydrogen

In [31]:
# Variables
c, carb, ester, eth, alip, defe, c_nonar,aro_ring, methyl, ali_chain, hydro, aro, nitro_groups, aniline, pyridin, quaternaryN, MW, oxy, hydroxyl, ali_chain_2, ali_chain_3, fluorine, sulfur, s1, s2, s3, s4, s5 , p1, p2, phosphorus= symbols('c carb ester eth aro_ring c_nonar alip defe methyl ali_chain hydro aro nitro_groups aniline pyridin quaternaryN MW oxy hydroxyl ali_chain_2 ali_chain_3 fluorine s1 s2 s3 s4 s5 sulfur p1 p2 phosphorus')
# Equations
equations = [
    
    # Carbon
    Eq(c_nonar, (defe+methyl+ali_chain*2+ali_chain_2*3+ali_chain_3*4+carb+ester)),# Non-aromatic carbons in the model
    Eq(defe,Defect*c*(1+q)),
    Eq(c,(c_nonar+aro_ring)),
    Eq(aro,(aro_ring-aniline-ester-carb-eth-fluorine-hydroxyl*0.3)), # Actual number of aromatic carbons in the model
    Eq(aro_ring,(c_aromatic-eth-pyridin-quaternaryN)),            # Aromatic carbons removed
    Eq(carb, Carbonyl*c*(0.95-z)),                                # Change: aromatic H
    Eq(ester, Ester*c*(0.95-z)),                                  # Change: aromatic H
    Eq(eth,0.5*Ether*c*(0.95-z)),                                 # Change: aromatic C-H (In multi CP-NMR ether groups count two carbons per oxygen)
    Eq(alip, Aliphatic*c*(1-h)),            
    Eq(methyl,(alip*u)/1),                                        # Change: aromatic H
    Eq(ali_chain,(alip*b)/2),                                     # Change: aromatic H
    Eq(ali_chain_2,(alip*p)/3),                                   # Change: aromatic H
    Eq(ali_chain_3,(alip*k)/4),                                   # Change: aromatic H
    
    # Nitrogen
    Eq(nitro_groups,(MW*Nitrogen)/14.0067*g),
    Eq(aniline,nitro_groups*2/4),                                  # Change: aromatic H
    Eq(pyridin,nitro_groups*1/4),                                  # Change: aromatic C-H
    Eq(quaternaryN,nitro_groups*1/4),                              # Change: interior C
    
    # Hydrogen
    Eq(hydro,(h_total+methyl*2+ali_chain*4+ali_chain_2*6 + ali_chain_3*4 +nitro_groups*3/4-carb-fluorine-s3-2*(s5+s1+s2)+p1+p2*2)),
    
    # Oxygen
    Eq(hydroxyl,((MW*Oxygen)/15.999)*v*0.9),                      # Change: aromatic H
    Eq(oxy,hydroxyl+ester*2+carb+eth+3*s4+2*s3+s5+4*p1),

    # Fluorine
    Eq(fluorine,(MW*Fluorine)/18.998),                             # Change: aromatic H

    # Sulfur
    Eq(sulfur,(MW*Sulfur)/32.065),                             
    Eq(s1,sulfur*Thiophene),                                      # Change: –CH2– or –CH–
    Eq(s2,sulfur*Thioether),                                      # Change: –CH2– or –CH–
    Eq(s3, sulfur*Sulfonyl),                                      # Change: –CH2– or –CH–
    Eq(s4, sulfur*Sulfo),                                         # Change: aromatic H
    Eq(s5, sulfur*Sulfinyl),                                      # Change: –CH2– or –CH–

    # Phosphorus
    Eq(phosphorus,(MW*Phosphorus)/30.974*g),          
    Eq(p1,phosphorus*Phosphate),                                  # Change: H
    Eq(p2,phosphorus*Phosphite),                                  # Change: H
    
    # Molecular weight
    Eq(MW,(c*12.011+hydro*1.00784+nitro_groups*14.0067+oxy*15.999+fluorine*18.998+sulfur*32.065+phosphorus*30.974))
     
]

# Solve the equations
solution = solve(equations, (c, carb, ester, eth, alip, c_nonar ,defe, aro_ring,methyl, ali_chain, hydro, aro, nitro_groups, aniline,pyridin, quaternaryN, MW, oxy, hydroxyl, ali_chain_2, ali_chain_3, fluorine,  sulfur, s1, s2, s3, s4, s5, p1, p2, phosphorus))

# Print the solutions
print("aro =", round(solution[aro]))
print("aro_ring =", round(solution[aro_ring]))
print("carb =", round(solution[carb]))
print("ester =", round(solution[ester]))
print("eth =", round(solution[eth]))
print("alip =", math.ceil(solution[alip]))
print("defe =", round(solution[defe]))
print("c_nonar =", round(solution[c_nonar]))
print("methyl =", round(solution[methyl]))
print("ali_chain =", math.ceil(solution[ali_chain]))
print("ali_chain_2 =", math.ceil(solution[ali_chain_2]))
print("ali_chain_3 =", math.ceil(solution[ali_chain_3]))
print("aniline =", round(solution[aniline]))
print("pyridin =",  math.ceil(solution[pyridin]))
print("quaternaryN =", round(solution[quaternaryN]))
print("Hydroxyl =", round(solution[hydroxyl]))
print("Thiophene =", round(solution[s1]))
print("Thioether =", round(solution[s2]))
print("Sulfonyl =", round(solution[s3]))
print("Sulfo =", round(solution[s4]))
print("Sulfinyl =", round(solution[s5]))
print("Phosphate =", round(solution[p1]))
print("Phosphite =", round(solution[p2]))       
print("MW =", round(solution[MW]))
print("Carbon =", round(solution[c]))
print("Hydrogen =", round(solution[hydro]))
print("Oxygen =", round(solution[oxy]))
print("Nitrogen =", round(solution[aniline])+math.ceil(solution[pyridin])+round(solution[quaternaryN]))
print("Fluorine =", round(solution[fluorine]))
print("Sulfur =", round(solution[s1])+round(solution[s2])+round(solution[s3])+round(solution[s4])+round(solution[s5]))
print("Phosphorus =", round(solution[p1])+round(solution[p2]))

aro = 4583
aro_ring = 5331
carb = 0
ester = 70
eth = 384
alip = 952
defe = 488
c_nonar = 1510
methyl = 0
ali_chain = 86
ali_chain_2 = 261
ali_chain_3 = 0
aniline = 12
pyridin = 7
quaternaryN = 6
Hydroxyl = 483
Thiophene = 0
Thioether = 0
Sulfonyl = 0
Sulfo = 0
Sulfinyl = 0
Phosphate = 0
Phosphite = 0
MW = 105842
Carbon = 6840
Hydrogen = 4605
Oxygen = 1006
Nitrogen = 25
Fluorine = 136
Sulfur = 0
Phosphorus = 0


### Create "holes" within the structure

In [32]:
if holes == 0:
    Internal_C_to_remove=0
else:
    Internal_C_to_remove = round(c_defect - solution[defe])
    print("Carbon to remove:", round(Internal_C_to_remove))

Carbon to remove: 51


In [33]:
# Adjust the excess carbon and create more realistic defects in the aromatic structures
if Internal_C_to_remove >= (num_objects*0.9):
    Internal_C_to_remove_Aro = math.ceil(Internal_C_to_remove*0.9)
else:
    Internal_C_to_remove_Aro=Internal_C_to_remove
print("Carbon to remove in aromatic structures:", Internal_C_to_remove_Aro)

Carbon to remove in aromatic structures: 51


### Define cross-links by fixing H/C ratio

In [34]:
def calculate_Err_H_C(i):
    mw = (round(solution[c])-Internal_C_to_remove)* 12.011 + i * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + round(solution[oxy])* 15.999 + round(solution[fluorine]) * 18.9984 + round(solution[sulfur]) * 32.065 + round(solution[phosphorus]) * 30.974 
    H_C_mod = i/round(solution[c]-Internal_C_to_remove)
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(round(system_size)):  
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Hydrogen needed:", best_i)
print("Error:", round(min_error,3))

Hydrogen needed: 4820
Error: 0.004


Keep in mind that the value of cross-links should be less than half of the total clusters in the system. For example, if you have 100 clusters, the value should be lower than 50. The cross-link value would also be adjusted later in the program because more hydrogen will be included at the end of the modification process due to changing the valence of bonds after the inclusion of the functional groups. 

In [35]:
Hydrogen_to_remove=round(solution[hydro])-best_i
if Hydrogen_to_remove < 0:
      Hydrogen_to_remove = 0
# Estimation of the hydrogen that will be added at the end of the modifications due to the change in valence of the new bonds
Hydroadded=round(solution[carb])+round(solution[hydroxyl])+round(solution[ester])+round(solution[ali_chain_2])+ round(solution[ali_chain])+round(solution[methyl]) 
newhydro=round(solution[hydro]- Hydrogen_to_remove + Hydroadded*0.1)
print("Hydrogen to remove:", round(Hydrogen_to_remove + Hydroadded*0.1))
crosslinks=round((Hydrogen_to_remove + Hydroadded*0.1)/2)
print("crosslinks:", crosslinks)
newMW = (round(solution[c])-Internal_C_to_remove)* 12.011 + best_i * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + round(solution[oxy]) * 15.999 + round(solution[sulfur]) * 32.065 + round(solution[fluorine])*18.9984 + round(solution[phosphorus]) * 30.974 

Hydrogen to remove: 90
crosslinks: 45


### See if you have an excess of oxygen


In [36]:
def calculate_Err_O(j):
    mw = (solution[c]-Internal_C_to_remove)* 12.011 + best_i * 1.00784 + ((solution[aniline] +solution[pyridin] +solution[quaternaryN])* 14.0067) + j * 15.999 + round(solution[fluorine])*18.9984 + round(solution[sulfur]) * 32.065 + round(solution[phosphorus]) * 30.974  
    OxygenMoles = (j * 15.999) / mw
    Err_O = (abs(OxygenMoles - Oxygen) / Oxygen)*100
    return Err_O

# Iterate over possible values of i and find the minimum error for O/C ratio
min_error = np.inf
best_j = None
for j in range(round(system_size)): 
    error = calculate_Err_O(j)
    if error < min_error:
        min_error = error
        best_j = j

print("Oxygen needed:", best_j)
print("Error:", round(min_error,3))

Oxygen needed: 1015
Error: 0.007


### Before modifying check: CNMR data

In [37]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove))-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.668
Error: 1.828


In [38]:
# Carbonyl Carbon
Err_Car=(abs((solution[carb]/(solution[c]-Internal_C_to_remove))-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon",round(solution[carb]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Car,3))

Carbonyl Carbon 0
Error: nan


In [39]:
# Carboxyl and Ester Carbon
Err_Carb=(abs((solution[ester]/(solution[c]-Internal_C_to_remove))-Ester)/Ester)*100
print("Ester Carbon",round(solution[ester]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Carb,3))

Ester Carbon 0.010
Error: 2.766


In [40]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(solution[c]-Internal_C_to_remove))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.113
Error: 2.766


In [41]:
# Aliphatic Carbon
Err_ali=(abs((solution[alip]/(solution[c]-Internal_C_to_remove))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round(solution[alip]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.140
Error: 7.804


In [42]:
# Defect Carbon
Err_defe=(abs((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove)-Defect))/Defect*100
print("Defect Carbon",round((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_defe,3))

Defect Carbon 0.072
Error: 2.680


### Before modifying check: Elemental composition 

In [43]:
# Carbon
Elem_c=((solution[c]-Internal_C_to_remove)*12.011)/newMW
print("Carbon:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Error:",round(Err_elemc,3))

Carbon: 0.7736
Error: 1.854


In [44]:
# Nitrogen
Elem_n=(solution[nitro_groups]*14.0067)/newMW
print("Nitrogen:",round(Elem_n,4))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Error:",round(Err_elemn,3))

Nitrogen: 0.0033
Error: 0.405


In [45]:
# Oxygen
Elem_o=((round(solution[oxy]))*15.999)/newMW
print("Oxygen:",round(Elem_o,4))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Error:",round(Err_elemo,3))

Oxygen: 0.1527
Error: 0.727


In [46]:
# Hydrogen
Elem_h=(best_i*1.00784)/newMW
print("Hydrogen:",round(Elem_h,4))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Error:",round(Err_elemh,3))

Hydrogen: 0.0461
Error: 1.952


In [47]:
# Fluorine
Elem_f=(round(solution[fluorine])*18.9984)/newMW
print("Fluorine:",round(Elem_f,4))
if Elem_f==0:
    Err_elemf=0
else:
    Err_elemf=(abs(Elem_f-Fluorine)/Fluorine)*100
print("Error:",round(Err_elemf,3))

Fluorine: 0.0245
Error: 0.043


In [48]:
# Sulfur
Elem_s=(round(solution[sulfur])*32.06)/newMW
print("Sulfur:",round(Elem_s,4))
if Elem_s==0:
    Err_elems=0
else:
    Err_elems=(abs(Elem_s-Sulfur)/Sulfur)*100
print("Error:",round(Err_elems,3))

Sulfur: 0
Error: 0


In [49]:
# Phosphorus
Elem_p=(round(solution[phosphorus])*30.974)/newMW
print("Phosphorus:",round(Elem_p,3))
if Elem_p==0:
    Err_elemp=0
else:
    Err_elemp=(abs(Elem_p-Phosphorus)/Phosphorus)*100
print("Error:",round(Err_elemp,3))

Phosphorus: 0
Error: 0


## Step 4: Modify structures and include functional groups

In [50]:
object_masses= builder.Get_Object_Masses()
object_names = list(object_masses.keys())
while not object_names:
    object_masses = builder.Get_Object_Masses()
    object_names = list(object_masses.keys())

### Include Ether Groups

In [51]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_totali= atom_counts["C"]
h_totali= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6266
Element H: 2820
Element N: 0
Element O: 0
Element S: 0
Element P: 0
Element F: 0


In [52]:
object_names = list(object_masses.keys())
modified_ether = []

if Biochar_Temp == 400:
    k1 = 0.8
elif Biochar_Temp == 500:
    k1 = 0.85
else:
    k1 = 1

num_molecules = round(solution[eth]*k1)

excluded_prefixes = ["circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*60):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group 
    object_ether = builder.Change_Element_CtoO(mol_to_edit,value)
    modified_ether.append(object_ether)

In [53]:
object_names = list(object_masses.keys())
modified_ether = []

if Biochar_Temp == 400:
    k = 0.2
elif Biochar_Temp == 500:
    k = 0.15
else:
    k = 0

num_molecules = round(solution[eth]*k)

excluded_prefixes = ["phenalene","circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*22):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")
    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group 
    object_ether = builder.Change_Element_CtoO(mol_to_edit,value)
    modified_ether.append(object_ether)

In [54]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]
o_total= atom_counts["O"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5882
Element H: 2428
Element N: 0
Element O: 384
Element S: 0
Element P: 0
Element F: 0


#### Check these values: Should be zero


In [55]:
cmd.select("carbon_bonded_to_oxygen", "elem O and neighbor elem O")

0

In [56]:
check_ether_o=o_total-round(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_o)
check_ether_c=c_totali-c_total-round(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_c)

0
0


###  Include Pyridinic Group 

In [57]:
object_names = list(object_masses.keys())
modified_pyridin = []

num_molecules = math.ceil(solution[pyridin])

excluded_prefixes = ["dibenzofuran","defect","benzo_a_fluorene", "benzo_b_fluorene","benzene","naphthalene","dibenzofuran","phenalene", "phenanthrene","anthracene","pyrene","benzo_g_h_i_perylene","benzo_a_pyrene","perylene","chrysene","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n'])<1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_2n'])
        if value_n in n_choices:
            break

    # Attach Pyridinic group 
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'elem="N"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'text_type="N.2"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'name="N"')

    # Store the modified Pyridinic
    modified_pyridin.append(object_pyridin)
    h_types = builder.examine_h(mol_to_edit)
    h_choice = random.choice(h_types['Aliphatic_N_inRing'])
    object_pyridin = cmd.remove(f'{mol_to_edit} and index {h_choice}')

    # Store the modified Pyridinic
    modified_pyridin.append(object_pyridin)

In [58]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_totali= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2421
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 0


#### Check these values: Should be zero


In [59]:
check_n=n_total-math.ceil(solution[pyridin])
print(check_n)

0


In [60]:
cmd.select("nitrogen_bonded_to_oxygen_nitrogen", "elem N and neighbor elem O")

0

In [61]:
cmd.select("nitrogen_bonded_nitrogen", "elem N and neighbor elem N")

0

### Include Fluorine 

In [62]:
object_names = list(object_masses.keys())
modified_fluorine = []

num_molecules = round(solution[fluorine])

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        f_types = builder.examine_main(mol_to_edit)

        if len(f_types['F_1n'])<=1:
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(filtered_object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Fluorine
        object_fluorine = builder.Attach_Fluorine(mol_to_edit, h_choice)
    modified_fluorine.append(object_fluorine)

In [63]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
f_total= atom_counts["F"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [64]:
cmd.do('colour')
check_f=f_total-round(solution[fluorine])
print(check_f)

0


In [65]:
cmd.select("fluorine_bonded_to_oxygen", "elem F and neighbor elem O")

0

In [66]:
cmd.select("fluorine_bonded_to_fluorine", "elem F and neighbor elem F")

0

### Include Sulfonyl

In [67]:
object_names = list(object_masses.keys())
modified_sulfonyl = []

num_molecules = round(solution[s3])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        ss_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_4n'])==0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
    
    cmd.select("carbon_not_bonded_to_O", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_O")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value = random.choice(ss_types['C_2n'])
        if value in c_choices:
            break

    # Attach Sulfur
    object_sulfonyl = builder.Change_Element_CtoS(mol_to_edit, value)
    modified_sulfonyl.append(object_sulfonyl)
    s_types = builder.examine_main(mol_to_edit)
    s_choice = s_types['S_2n']
    molecule = mol_to_edit
    
    for s_index in s_choice:
        
        cmd.select(f"at1_{s_index}", f"{mol_to_edit} & index {s_index}")
        
        for oxygen_number in range(2):  # Loop to add two oxygen atoms
            
            cmd.do(f'load ./functional_groups/O.mol2')
            cmd.select(f"at2_O{oxygen_number + 1}", "O & name O01")
        
            if cmd.count_atoms(f"at1_{s_index}") == 1 and cmd.count_atoms(f"at2_O{oxygen_number + 1}") == 1:
                # Edit, fuse, and bond
                cmd.do(f'edit at2_O{oxygen_number + 1}, at1_{s_index}')
                cmd.do('fuse')
                cmd.do(f'bond at1_{s_index} at2_O{oxygen_number + 1} type double')
                cmd.do('unpick')
                cmd.do('rebuild all')
                cmd.do(f'zoom {mol_to_edit}')
                cmd.do(f'clean {mol_to_edit}')
            else:
                print(f"Invalid selection(s) for index {s_index}")
            
            cmd.delete(f"at2_O{oxygen_number + 1}")
            cmd.delete("O")
    
    cmd.do('set sphere_scale, 0.2, (all)')
    cmd.do('set_bond stick_radius, 0.14, (all), (all)')
    cmd.do('show sticks')
    cmd.do('show spheres')
    modified_sulfonyl.append(object_sulfonyl)

In [68]:
for name in cmd.get_names("all"):
    if name.startswith("at1_"):
        cmd.delete(name)

In [69]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
s_total = atom_counts["S"]
o_total2 = atom_counts["O"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [70]:
check_s=s_total-round(solution[s3])
print(check_s)

0


In [71]:
check_o=o_total2-o_total-round(solution[s3]*2)
print(check_o)

0


In [72]:
cmd.select("sulfur_bonded_to_nitrogen", "elem S and neighbor elem N")

0

In [73]:
cmd.select("sulfur_bonded_to_sulfur", "elem S and neighbor elem S")

0

### Include Sulfinyl

In [74]:
object_names = list(object_masses.keys())
modified_sulfinyl = []

num_molecules =round(solution[s5])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        ss_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_3n']) ==0 and len(ss_types['S_4n']) ==0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
           
    cmd.select("carbon_not_bonded_to_S_O", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_S_O")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value = random.choice(ss_types['C_2n'])
        if value in c_choices:
            break

    # Attach Sulfur
    object_sulfinyl = builder.Change_Element_CtoS(mol_to_edit, value)
    modified_sulfinyl.append(object_sulfinyl)
    s_types = builder.examine_main(mol_to_edit)
    s_choice = s_types['S_2n']
    
    for s_index in s_choice:
        
        cmd.select(f"at1_{s_index}", f"{mol_to_edit} & index {s_index}")
        
        for oxygen_number in range(1):  # Loop to add one oxygen atom
            
            cmd.do(f'load ./functional_groups/O.mol2')
            cmd.select(f"at2_O{oxygen_number + 1}", "O & name O01")
        
            if cmd.count_atoms(f"at1_{s_index}") == 1 and cmd.count_atoms(f"at2_O{oxygen_number + 1}") == 1:
                # Edit, fuse, and bond
                cmd.do(f'edit at2_O{oxygen_number + 1}, at1_{s_index}')
                cmd.do('fuse')
                cmd.do(f'bond at1_{s_index} at2_O{oxygen_number + 1} type double')
                cmd.do('unpick')
                cmd.do('rebuild all')
                cmd.do(f'zoom {mol_to_edit}')
                cmd.do(f'clean {mol_to_edit}')
            else:
                print(f"Invalid selection(s) for index {s_index}")
            
            cmd.delete(f"at2_O{oxygen_number + 1}")
            cmd.delete("O")
    
    cmd.do('set sphere_scale, 0.2, (all)')
    cmd.do('set_bond stick_radius, 0.14, (all), (all)')
    cmd.do('show sticks')
    cmd.do('show spheres')
    modified_sulfinyl.append(object_sulfinyl)

In [75]:
for name in cmd.get_names("all"):
    if name.startswith("at1_"):
        cmd.delete(name)

In [76]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
s_total2 = atom_counts["S"]
o_total3 = atom_counts["O"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [77]:
check_s=s_total2-round(solution[s3])-round(solution[s5])
print(check_s)

0


In [78]:
check_o=o_total3-o_total2-round(solution[s3])
print(check_o)

0


In [79]:
cmd.select("sulfur_bonded_to_nitrogen", "elem S and neighbor elem N")

0

In [80]:
cmd.select("sulfur_bonded_to_sulfur", "elem S and neighbor elem S")

0

### Include Thiophene 

In [81]:
object_names = list(object_masses.keys())
modified_thiophene = []

num_molecules = round(solution[s1] * 0.6)

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        # Get available carbon types
        ss_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_1n']) <= 1 and len(ss_types['S_2n']) <= 4 and len(ss_types['S_3n']) <= 2 and len(ss_types['S_4n']) <= 1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("C_not_bonded_to_S_O_N", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem S) and not neighbor (elem N) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("C_not_bonded_to_S_O_N")
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    # Ensure a carbon atom is selected for thiophene attachment
    while True:
        value = random.choice(ss_types['C_2n'])
        if value in c_choices:
            break

    # Attach Thiophene group
    object_thiophene = builder.Change_Element_CtoS(mol_to_edit, value)
    modified_thiophene.append(object_thiophene)

In [82]:
object_names = list(object_masses.keys())
modified_thiophene = []

num_molecules = round(solution[s1]*0.4)

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]


if num_molecules > len(object_names):
    molecules_to_edit = random.choices(filtered_object_names , k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        ss_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_1n']) <= 1 and len(ss_types['S_3n']) <= 1 and len(ss_types['S_3n']) <= 1 and len(ss_types['S_4n']) <= 1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("C_not_bonded_to_S_O_N", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem S) and not neighbor (elem N) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("C_not_bonded_to_S_O_N")
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(ss_types['C_2n'])
        if value in c_choices:
            break

    # Attach Thiophene group 
    object_thiophene = builder.Change_Element_CtoS(mol_to_edit,value)
    modified_thiophene.append(object_thiophene)

In [83]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
s_total3= atom_counts["S"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [84]:
check_s=s_total3-round(solution[s1])-round(solution[s3])-round(solution[s5])
print(check_s)

0


In [85]:
cmd.select("sulfur_bonded_to_sulfur", "(elem S) and neighbor (elem S)")

0

### Include Sulfide

In [86]:
object_names = list(object_masses.keys())
modified_thioether = []

num_molecules = round(solution[s2])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        ss_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_1n'])<=1  and len(ss_types['S_2n']) <=1 and len(ss_types['S_3n']) <=1 and len(ss_types['S_4n']) <=1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
    
    cmd.select("C_not_bonded_to_S_O_N", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem S) and not neighbor (elem N) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("C_not_bonded_to_S_O_N")
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(ss_types['C_2n'])
        if value in c_choices:
            break

    # Attach Thioether group 
    object_thioether = builder.Change_Element_CtoS(mol_to_edit,value)
    modified_thioether.append(object_thioether)

In [87]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
s_total4= atom_counts["S"]
o_total4 = atom_counts["O"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [88]:
check_s=s_total4-round(solution[s1])-round(solution[s2])-round(solution[s3])-round(solution[s5])
print(check_s)

0


### Include Sulfo

In [89]:
object_names = list(object_masses.keys())
modified_sulfo = []

num_molecules = round(solution[s4])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        s_types = builder.examine_main(mol_to_edit)

        if len(ss_types['S_1n'])<=1  and len(ss_types['S_2n']) <=1 and len(ss_types['S_3n']) <=1 and len(ss_types['S_4n']) <=1:
           break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Sulfo
        object_sulfo = builder.Attach_SO3H(mol_to_edit, h_choice)
        modified_sulfo.append(object_sulfo)

In [90]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
s_total5 = atom_counts["S"]
o_total5 = atom_counts["O"]
h_total5 = atom_counts["H"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 384
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [91]:
check_s=s_total5-round(solution[s1])-round(solution[s2])-round(solution[s3])-round(solution[s4])-round(solution[s5])
print(check_s)

0


In [92]:
check_o=o_total5-o_total4-round(solution[s4])*3
print(check_o)

0


In [93]:
cmd.select("sulfur_bonded_to_nitrogen", "elem S and neighbor elem N")

0

In [94]:
cmd.select("sulfur_bonded_to_sulfur", "elem S and neighbor elem S")

0

### Include Hydroxyl Groups

In [95]:
object_names = list(object_masses.keys())
modified_hydroxyls = []

num_molecules = round(solution[hydroxyl]/2)

excluded_prefixes = ["phenalene"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        h_types = builder.examine_h(mol_to_edit)
        
        if len(o_types['O_2n'])<round(O_C*40) and len(o_types['O_1n'])< round(O_C*40) and len(h_types['Aliphatic_C_inRing'])>=1:
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(filtered_object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Hydroxyl group
        object_hydroxyl = builder.Attach_OH(mol_to_edit, h_choice)
    modified_hydroxyls.append(object_hydroxyl)

In [96]:
object_names = list(object_masses.keys())
modified_hydroxyls = []

num_molecules = round(solution[hydroxyl]/2)

excluded_prefixes = ["phenalene"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        h_types = builder.examine_h(mol_to_edit)
        
        if len(o_types['O_2n'])<round(O_C*50) and len(o_types['O_1n'])< round(O_C*50) and len(h_types['Aliphatic_C_inRing'])>=1:
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(filtered_object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Hydroxyl group
        object_hydroxyl = builder.Attach_OH(mol_to_edit, h_choice)
    modified_hydroxyls.append(object_hydroxyl)

In [97]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total= atom_counts["O"]
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 866
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [98]:
check_ox=o_total-(o_total5+round(solution[hydroxyl]/2)+round(solution[hydroxyl]/2))
print(check_ox)

0


### Include Carbonyl Groups

In [99]:
object_names = list(object_masses.keys())
modified_carbonyl = []

num_molecules = round(solution[carb])

excluded_prefixes = ["benzo_a_fluorene", "benzo_b_fluorene","phenalene", "phenanthrene","anthracene","pyrene","benzo_a_pyrene","perylene","chrysene","benzo_b_fluoranthene","N5"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


for mol_to_edit in molecules_to_edit:
    while True:

        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")
            
        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*55) and len(o_types['O_1n'])<round(O_C*55):
            break  # Condition met, proceed to modify the molecule
            
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit) 

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Carbonyl group
    object_carbonyl = builder.Attach_Carbonyl(mol_to_edit, h_choice)
    modified_carbonyl.append(object_carbonyl)

In [100]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new= atom_counts["O"]
c_total_new= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 5875
Element H: 2285
Element N: 7
Element O: 866
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [101]:
check_c=c_total_new-(c_total+round(solution[carb]))
print(check_c)
check_o=o_total_new-(o_total+round(solution[carb]))
print(check_o)

0
0


### Include Ester and Carboxyl Groups

In [102]:
object_names = list(object_masses.keys())
modified_ester = []

num_molecules = math.floor(solution[ester]*0.9) # 90% as ester groups

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:
            
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if Biochar_Temp==700:
            if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*80) and len(o_types['O_1n'])<round(O_C*80):
                break  # Condition met, proceed to modify the molecule
        else:
             if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*65) and len(o_types['O_1n'])<round(O_C*65):
                break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing']) == 0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Ester group
    object_ester = builder.Attach_Ester(mol_to_edit, h_choice)
    modified_ester.append(object_ester)

In [103]:
object_names = list(object_masses.keys())
modified_carbox = []

num_molecules = math.ceil(solution[ester]*0.1) # 10% as carboxylic groups

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:
            
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if Biochar_Temp==700:
            if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*80) and len(o_types['O_1n'])<round(O_C*80):
                break  # Condition met, proceed to modify the molecule
        else:
             if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*70) and len(o_types['O_1n'])<round(O_C*70):
                break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing']) == 0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Carboxylic group
    object_carbox = builder.Attach_Carbox(mol_to_edit, h_choice)
    modified_carbox.append(object_carbox)

In [104]:
cmd.do("valence guess, all")
def add_hydrogens_excluding_prefixes():
    excluded_prefixes = []
    
    filtered_object_names = [
        name for name in object_names
        if not any(name.startswith(prefix) for prefix in excluded_prefixes)
    ]
    
    for name in filtered_object_names:
        cmd.h_add(name)
        
add_hydrogens_excluding_prefixes()

In [105]:
for obj in object_names:
    cmd.do('clean %s' % obj)

In [106]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new2= atom_counts["O"]
c_total_new2= atom_counts["C"]
h_total_new2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6037
Element H: 2892
Element N: 7
Element O: 1010
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [107]:
check_o=o_total_new2-(o_total_new+math.ceil(solution[ester]*0.1)*2+math.floor(solution[ester]*0.9)*2)
print(check_o)

6


### Include Methyl Groups

In [108]:
object_names = list(object_masses.keys())
modified_methyl = []

num_molecules = round(solution[methyl])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:  

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>6:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break
            
    # Attach Methyl group 
    object_methyl = builder.Attach_CH3(mol_to_edit, h_choice)
    modified_methyl.append(object_methyl)

In [109]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_new= atom_counts["C"]
h_total_new= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6037
Element H: 2892
Element N: 7
Element O: 1010
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [110]:
check_c=c_total_new-(c_total_new2+round(solution[methyl]))
print(check_c)
check_hy=h_total_new-(h_total_new2+round(solution[methyl])*2)
print(check_hy)

0
0


### Include CH2CH3 Groups

In [111]:
object_names = list(object_masses.keys())
modified_ali_chain = []

num_molecules = round(solution[ali_chain])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>8:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Aliphatic group           
    object_ali_chain = builder.Attach_CH2CH3(mol_to_edit, h_choice)
    modified_ali_chain.append(object_ali_chain)

In [112]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6116
Element H: 3102
Element N: 7
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


### Include (CH2)2-CH3 Groups

In [114]:
object_names = list(object_masses.keys())
modified_ali_chain_2 = []

num_molecules = round(solution[ali_chain_2]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>3:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)
    modified_ali_chain_2.append(object_ali_chain2)

In [115]:
object_names = list(object_masses.keys())
modified_ali_chain_2 = []

num_molecules = round(solution[ali_chain_2]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>11:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)
    modified_ali_chain_2.append(object_ali_chain2)

In [116]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_2= atom_counts["C"]
h_total_2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6896
Element H: 4662
Element N: 7
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


In [117]:
cmd.do("valence guess, all")

In [118]:
def add_hydrogens_excluding_prefixes():
    excluded_prefixes = []
    
    filtered_object_names = [
        name for name in object_names
        if not any(name.startswith(prefix) for prefix in excluded_prefixes)
    ]
    
    for name in filtered_object_names:
        cmd.h_add(name)
        
add_hydrogens_excluding_prefixes()

In [119]:
for obj in object_names:
    cmd.do('clean %s' % obj)

#### Check these values: Should be zero

In [120]:
check_c=c_total_2-(c_total+round(solution[ali_chain_2]/2)*3+round(solution[ali_chain_2]/2)*3)
print(check_c)

0


### Include NH3 Groups

In [121]:
object_names = list(object_masses.keys())
modified_Aniline = []

num_molecules = round(solution[aniline])

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:           
    while True:

        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(n_types['N_2n'])==0 and len(h_choices)>1 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Aniline group
    object_Aniline = builder.Attach_Aniline(mol_to_edit, h_choice)
    modified_Aniline.append(object_Aniline)

In [122]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_total_new= atom_counts["H"]
c_total_new= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6896
Element H: 4934
Element N: 19
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


In [123]:
cmd.do("valence guess, all")

In [124]:
def add_hydrogens_excluding_prefixes():
    excluded_prefixes = []
    
    filtered_object_names = [
        name for name in object_names
        if not any(name.startswith(prefix) for prefix in excluded_prefixes)
    ]
    
    for name in filtered_object_names:
        cmd.h_add(name)
        
add_hydrogens_excluding_prefixes()

In [125]:
for obj in object_names:
    cmd.do('clean %s' % obj)

#### Check these values: Should be zero

In [126]:
check_n=n_total-(round(solution[aniline])+ math.ceil(solution[pyridin]))
print(check_n)

0


### Include Quaternary Nitrogen

In [127]:
object_names = list(object_masses.keys())
modified_quater = []

num_molecules = round(solution[quaternaryN])

excluded_prefixes = ["circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n']) == 0 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
            
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_3n'])
        if value_n in n_choices:
            break
    # Change the Carbon selected to Nitrogen
    object_quater = builder.Change_Element_CtoN(mol_to_edit,value_n)
    modified_quater.append(object_quater)

In [128]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total_neww= atom_counts["N"]
c_total_neww= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5050
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero


In [129]:
check_cc=c_total_new-(round(solution[quaternaryN])+c_total_neww)
print(check_cc)
check_n=n_total_neww-(n_total+round(solution[quaternaryN]))
print(check_n)

0
0


### Include Phosphate

In [130]:
object_names = list(object_masses.keys())
modified_phosphate = []

num_molecules = round(solution[p1])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        f_types = builder.examine_main(mol_to_edit)
        h_types = builder.examine_h(mol_to_edit)

        # Modify the condition to include the OR logic for Aliphatic carbon types
        if len(f_types['P_4n']) <= 2 and (len(h_types['Aliphatic_C_inRing']) >= 1 or len(h_types['Aliphatic_C']) >= 1):
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

    # Attach Phosphate
    object_phosphate = builder.Attach_phosphate(mol_to_edit, h_choice)
    modified_phosphate.append(object_phosphate)


In [131]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
p_total= atom_counts["P"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5050
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [132]:
cmd.do('colour')
check_p=p_total-round(solution[p1])
print(check_p)

0


In [133]:
cmd.select("P_bonded_to_P", "elem P and neighbor elem P")

0

### Include Phosphine

In [134]:
object_names = list(object_masses.keys())
modified_phosphine = []

num_molecules = round(solution[p2])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        f_types = builder.examine_main(mol_to_edit)
        h_types = builder.examine_h(mol_to_edit)

        if len(f_types['P_4n'])<=5 and (len(h_types['Aliphatic_C_inRing'])>= 1 or len(h_types['Aliphatic_C'])>=1):
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Phosphine
        object_phosphine = builder.Attach_phosphine(mol_to_edit, h_choice)
    modified_phosphine.append(object_phosphine)

In [135]:
def add_hydrogens_excluding_prefixes():
    excluded_prefixes = []
    
    filtered_object_names = [
        name for name in object_names
        if not any(name.startswith(prefix) for prefix in excluded_prefixes)
    ]
    
    for name in filtered_object_names:
        cmd.h_add(name)
        
add_hydrogens_excluding_prefixes()

In [136]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
p_total2= atom_counts["P"]
c_total=atom_counts["C"]
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5050
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [137]:
cmd.do('colour')
check_p=p_total2-round(solution[p1])-round(solution[p2])
print(check_p)

0


In [138]:
cmd.select("P_bonded_to_P", "elem P and neighbor elem P")

0

### Include (CH)3-CH2 Groups

In [139]:
object_names = list(object_masses.keys())
modified_ali_chain_3 = []

num_molecules = round(solution[ali_chain_3]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>3:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain3 = builder.Attach_C4H4(mol_to_edit, h_choice)
    modified_ali_chain_3.append(object_ali_chain3)

In [140]:
object_names = list(object_masses.keys())
modified_ali_chain_3 = []

num_molecules = round(solution[ali_chain_3]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>3:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain3 = builder.Attach_C4H4(mol_to_edit, h_choice)
    modified_ali_chain_3.append(object_ali_chain3)

In [141]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_22= atom_counts["C"]
h_total_22= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5050
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check these values: Should be zero

In [142]:
check_c=c_total_22-(c_total + round(solution[ali_chain_3]/2)*4 + round(solution[ali_chain_3]/2)*4)
print(check_c)

0


### Correct atom valence and include hydrogens to fix the inclusion of functional groups

In [143]:
cmd.do('colour')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')
cmd.bg_color("white")
cmd.set("ray_shadows", "off")

In [157]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC1= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
FinalN= atom_counts["N"]
FinalS= atom_counts["S"]
FinalP= atom_counts["P"]
FinalF= atom_counts["F"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5050
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


In [158]:
cmd.do("valence guess, all")

In [159]:
object_names = list(object_masses.keys())
for obj in object_names:
    cmd.do('clean %s' % obj)

In [160]:
builder.clear_label()

## Step 5: Create a cross-linked network to fix H/C ratio

### Obtain the molecular formula of decorated clusters before crosslinking

In [161]:
object_list = cmd.get_object_list()
def get_atom_counts(object_list):
    elements = ['C', 'H', 'N', 'O', 'S', 'P', 'F']
    counts = [object_list]  # Start with the molecule name
    for element in elements:
        count = cmd.count_atoms(f'elem {element} and model {object_list}')
        counts.append(count)
    return counts

with open('Molecular_formula_before_crosslinking.txt', 'w') as file:
    file.write("Molecule,C,H,N,O,S,P,F\n")
    
    for molecule in object_list:
        atom_counts = get_atom_counts(molecule)
        file.write(",".join(map(str, atom_counts)) + '\n')

print("Composition analysis complete. Results saved to 'Molecular_formula_before_crosslinking.txt'.")

Composition analysis complete. Results saved to 'Molecular_formula_before_crosslinking.txt'.


### Check againg cross-links by fixing H/C ratio

In [164]:
def calculate_Err_H_C(i):
    H_C_mod = i/(FinalC1-Internal_C_to_remove)
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(10000):  # Adjust the range as needed based on how many carbons you have in the simulation
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Best i value:", best_i)
print("Minimum Err_H_C:", round(min_error,3))

Best i value: 4856
Minimum Err_H_C: 0.006


#### The number of cross-links needs to be an even number

In [165]:
def round_to_lowest_even(number):
    divided_number = number / 2
    rounded_number = math.floor(divided_number)
    lowest_even_number = rounded_number * 2
    return lowest_even_number

In [166]:
Hydrogen_to_remove=round_to_lowest_even(FinalH-best_i)
print("Hydrogen to be removed:",Hydrogen_to_remove)
crosslinks=round_to_lowest_even(Hydrogen_to_remove/2)
print("Cross-links needed:",crosslinks)
newhydro=FinalH-Hydrogen_to_remove
newMW=(FinalC1-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + FinalN* 14.0067 + FinalO * 15.999 + FinalS * 32.06 + FinalP * 30.974 + FinalF * 18.998

Hydrogen to be removed: 194
Cross-links needed: 96


In [167]:
object_list = cmd.get_object_list()
num_objects = len(object_list)
if crosslinks >= (num_objects*0.75):
    crosslinks=round_to_lowest_even(crosslinks*0.65)
elif crosslinks <=0 or crosslinks < 10:
    crosslinks=round_to_lowest_even(FinalH*0.01)
else:
    crosslinks=crosslinks
print("Cross-links needed:",crosslinks)
newhydro=FinalH-crosslinks*2

Cross-links needed: 62


In [168]:
HydrogenMoles = (newhydro * 1.00784) / newMW
CarbonMoles = ((FinalC1-Internal_C_to_remove) * 12.011) / newMW
H_C_mod = (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C ratio:",round(H_C_mod,3))
Bridgehead_mod =1- (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(Bridgehead_mod,3))

H/C ratio: 0.720
Bridgehead Carbon: 0.280


### This step is divide 6 times to create a diverse distribution

In [169]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

### Step one: 25% of the total cross-links

In [170]:
excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names, round_to_lowest_even(crosslinks/2))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

30


In [171]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [172]:
modified_cross = [None] * round_to_lowest_even(crosslinks/4)
mol_to_edit = [None] * round_to_lowest_even(crosslinks/2)
h_types = [None] * round_to_lowest_even(crosslinks/2)
h_choice = [None] *round_to_lowest_even(crosslinks/2)

for i in range(0,  round_to_lowest_even(crosslinks/4)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/4), round_to_lowest_even(crosslinks/2)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [173]:
for k in range(0, round_to_lowest_even(crosslinks/4)):
    modified_cross[k]= builder.Crosslink_mols1(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/4)], h_choice[k+round_to_lowest_even(crosslinks/4)])

In [174]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC_1= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 5022
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero


In [175]:
Totalc=FinalC_1-FinalC1
print(Totalc)

0


### Step two: + 25% of the total cross-links

In [176]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [177]:
excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names, round_to_lowest_even(crosslinks/2))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

30


In [178]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [179]:
modified_cross = [None] * round_to_lowest_even(crosslinks/4)
mol_to_edit = [None] * round_to_lowest_even(crosslinks/2)
h_types = [None] * round_to_lowest_even(crosslinks/2)
h_choice = [None] *round_to_lowest_even(crosslinks/2)

for i in range(0,  round_to_lowest_even(crosslinks/4)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/4), round_to_lowest_even(crosslinks/2)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [180]:
for k in range(0, round_to_lowest_even(crosslinks/4)):
    modified_cross[k]= builder.Crosslink_mols2(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/4)], h_choice[k+round_to_lowest_even(crosslinks/4)])

In [181]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC2= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 4994
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero

In [182]:
Totalc=FinalC2-FinalC1
print(Totalc)

0


### Step three: + 12.5% of the total cross-links

In [183]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [184]:
object_names_new = list(object_masses.keys())
excluded_prefixes = ["defect","combined"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

14


In [185]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [186]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [187]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols3(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [188]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC3= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 4982
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero

In [189]:
Totalc2=FinalC3-FinalC1
print(Totalc2)

0


### Step four: + 12.5% of the total cross-links

In [190]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [191]:
print(len(list(object_masses.keys())))

74


In [192]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

14


In [193]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [194]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [195]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols4(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [196]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC4= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 4970
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero

In [197]:
Totalc3=FinalC4-FinalC1
print(Totalc3)

0


### Step five: + 12.5% of the total cross-links

In [198]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [199]:
print(len(list(object_masses.keys())))

68


In [200]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

14


In [201]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [202]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [203]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols5(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [204]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC5= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 4958
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero

In [205]:
Totalc4=FinalC5-FinalC1
print(Totalc4)

0


### Step six: final 12.5% of the total cross-links

In [206]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [207]:
print(len(list(object_masses.keys())))

62


In [208]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

14


In [209]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [210]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [211]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols6(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

### Create holes: Aromatic structures

In [212]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [213]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
Final_C1= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6890
Element H: 4946
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


In [214]:
modified_hole = []

excluded_prefixes = ["defect", "fluorine", "phenalene", "phenanthrene", "anthracene", "tetracene", "pentacene", "pyrene", "chrysene", "benzo_a_fluorene", "benzo_b_fluoranthene", "benzo_b_fluorene", "coronene", "benzo_a_pyrene", "benzo_g_h_i_perylene", "m"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Initialize a list to keep track of tried molecules
tried_molecules = []

while True:
    num_molecules = round(Internal_C_to_remove - check_c)
    # Reset the list of modified holes and tried molecules for each iteration
    modified_hole = []
    tried_molecules = []

    # Prepare the list of molecules to edit
    if num_molecules > len(filtered_object_names):
        molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
    else:
        molecules_to_edit = random.sample(filtered_object_names, num_molecules)

    while molecules_to_edit:
        mol_to_edit = molecules_to_edit.pop(0)  # Get the first molecule from the list

        if mol_to_edit in tried_molecules:
            continue  # Skip if we've already tried this molecule
        tried_molecules.append(mol_to_edit)

        c_types = builder.examine_main(mol_to_edit)
        cmd.select("C-C-C", f"({mol_to_edit} and elem C and not neighbor (elem H) and not neighbor (elem F) and not neighbor (elem S) and not neighbor (elem P) and not neighbor (elem O) and not neighbor ((elem C) and neighbor (elem H)))")
        
        # Get the selection of atoms
        selection_dict = cmd.get_model("C-C-C")
        c_3n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
        
        if c_3n_choices:  # Check if there are valid C_3n choices
            value_c = random.choice(c_types['C_3n'])
            if value_c in c_3n_choices:
                # Remove the internal carbon atom to create a hole in the structure
                object_hole = cmd.remove(f'{mol_to_edit} and index {value_c}')
                modified_hole.append(object_hole)
                print(f"Edited molecule: {mol_to_edit} at index {value_c}")
                continue  # Move to the next molecule in the list (if any)

        # If there are no valid choices or removal wasn't successful
        print(f"No valid C_3n choices or failed to edit for {mol_to_edit}.")
        
    if not modified_hole:
        print("No molecules were successfully edited.")
    
    # Count atoms
    atom_counts = {}
    elements = ["C", "H", "N", "O", "S", "P", "F"]

    for element in elements:
        count = cmd.select(f"elem {element}")
        atom_counts[element] = count
    Final_N = atom_counts["N"]
    Final_C = atom_counts["C"]
    Final_O = atom_counts["O"]
    Final_H = atom_counts["H"]
    Final_S = atom_counts["S"]
    Final_P = atom_counts["P"]
    Final_F = atom_counts["F"]

    for element, count in atom_counts.items():
        print(f"Element {element}: {count}")

    check_c = Final_C - (FinalC1 - Internal_C_to_remove)
    print(check_c)

    if check_c <= 0:
        print("Stopping loop as check_c is exactly 0.")
        break


No valid C_3n choices or failed to edit for combined_e5.
No valid C_3n choices or failed to edit for combined_b4.
No valid C_3n choices or failed to edit for combined_e2.
Edited molecule: pentatriacotaene54 at index 15
Edited molecule: combined_c3 at index 103
No valid C_3n choices or failed to edit for pentatriacotaene65.
No valid C_3n choices or failed to edit for pentatriacotaene25.
Edited molecule: pentatriacotaene27 at index 50
Edited molecule: pentatriacotaene66 at index 48
No valid C_3n choices or failed to edit for combined_d2.
No valid C_3n choices or failed to edit for pentatriacotaene24.
No valid C_3n choices or failed to edit for combined_a4.
No valid C_3n choices or failed to edit for combined_b1.
No valid C_3n choices or failed to edit for N106.
Edited molecule: N86 at index 3
No valid C_3n choices or failed to edit for combined_d1.
Edited molecule: pentatriacotaene39 at index 48
Edited molecule: pentatriacotaene1 at index 30
Edited molecule: pentatriacotaene33 at index 2

In [215]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [216]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "P", "F"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
Final_N= atom_counts["N"]
Final_C= atom_counts["C"]
Final_O= atom_counts["O"]
Final_H= atom_counts["H"]
Final_S= atom_counts["S"]
Final_P= atom_counts["P"]
Final_F= atom_counts["F"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6833
Element H: 4946
Element N: 25
Element O: 1004
Element S: 0
Element P: 0
Element F: 136


#### Check this value: Should be zero or close to zero

In [219]:
check_c=Final_C-(FinalC1-Internal_C_to_remove)
print(check_c)

-6


In [220]:
cmd.do("valence guess, all")

## Step 6: Check the final ultimate analysis

In [221]:
builder.clear_label()

In [222]:
newMW2=Final_C* 12.011 + Final_H * 1.00784 + Final_N* 14.0067 + Final_O * 15.999 +Final_S * 32.06 + Final_P * 30.974 + Final_F * 18.998
print("MW:",round(newMW2))

MW: 106053


### Ultimate Analysis

In [223]:
# Carbon
Elem_c=(Final_C*12.011)/newMW2
print("Total C:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Final Error:",round(Err_elemc,3))

Total C: 0.7739
Final Error: 1.818


In [224]:
# Nitrogen
Elem_n=(Final_N*14.0067)/newMW2
print("Total N:",round(Elem_n,5))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Final Error:",round(Err_elemn,3))

Total N: 0.0033
Final Error: 0.055


In [225]:
# Oxygen
Elem_o=(Final_O*15.999)/newMW2
print("Total O:",round(Elem_o,4))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Final Error:",round(Err_elemo,3))

Total O: 0.1515
Final Error: 1.52


In [226]:
# Hydrogen
Elem_h=Final_H*1.00784/newMW2
print("Total H:",round(Elem_h,4))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Final Error:",round(Err_elemh,3))

Total H: 0.047
Final Error: 0.006


In [227]:
# Fluorine
Elem_f=(Final_F*18.9984)/newMW2
print("Total F:",round(Elem_f,4))
if Elem_f==0:
    Err_elemf=0
else:
    Err_elemf=(abs(Elem_f-Fluorine)/Fluorine)*100
print("Error:",round(Err_elemf,3))

Total F: 0.0244
Error: 0.559


In [228]:
# Sulfur
Elem_s=(Final_S*32.06)/newMW2
print("Total S:",round(Elem_s,4))
if Elem_s==0:
    Err_elems=0
else:
    Err_elems=(abs(Elem_s-Sulfur)/Sulfur)*100
print("Final Error:",round(Err_elems,3))

Total S: 0.0
Final Error: 0


In [229]:
# Phosphorus
Elem_p=(Final_P*30.974)/newMW2
print("Total P:",round(Elem_p,3))
if Elem_p==0:
    Err_elemp=0
else:
    Err_elemp=(abs(Elem_p-Phosphorus)/Phosphorus)*100
print("Final Error:",round(Err_elemp,3))

Total P: 0.0
Final Error: 0


In [230]:
# H/C ratio
CarbonMoles = (Final_C * 12.011) / newMW2
HydrogenMoles = (Final_H * 1.00784) / newMW2
H_C_mod =(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C Ratio:",round(H_C_mod,4))
Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
print("Error:",round(Err_H_C,3))

H/C Ratio: 0.7238
Error: 1.949


In [231]:
# O/C ratio
OxygenMoles = (Final_O * 15.999) / newMW2
O_C_mod =1/((CarbonMoles * 15.999)/(OxygenMoles  * 12.011)) 
print("O/C Ratio:",round(O_C_mod,4))
Err_C_O = (abs(O_C_mod- O_C) / O_C) * 100
print("Error:",round(Err_C_O,3))

O/C Ratio: 0.1469
Error: 2.044


In [232]:
# Bridgehead carbon
brigd =1-(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(brigd,3))
Err_brigd = (abs(brigd - BridgheadCarbon) / BridgheadCarbon) * 100
print("Error:",round(Err_brigd,3))

Bridgehead Carbon: 0.276
Error: 4.772


In [233]:
cmd.save("biochar.pse")

## Step 7: Check the final 13C NMR 

### CNMR

In [234]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/Final_C)-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(Final_C)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.663
Error: 2.454


In [235]:
# Carbonyl Carbon
Err_Carb=(abs(((solution[carb])/Final_C)-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon:",round(((solution[carb])/(Final_C)),3))
print("Error:",round(Err_Carb,3))

Carbonyl Carbon: 0
Error: nan


In [236]:
# Carboxyl/Lactone/Ester Carbon
Err_Carbox=(abs(((solution[ester])/Final_C)-Ester)/Ester)*100
print("Ester Carbon:",round(((solution[ester])/(Final_C)),3))
print("Error:",round(Err_Carbox,3))

Ester Carbon: 0.010
Error: 2.112


In [237]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(Final_C))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(Final_C),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.112
Error: 2.112


In [238]:
# Aliphatic Carbon
Err_ali=(abs(((solution[alip])/(Final_C))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round((solution[alip])/(Final_C),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.139
Error: 7.117


In [239]:
# Defective Carbon
Err_def=(abs(((solution[defe])/(Final_C))-Defect)/Defect)*100
print("Defective Carbon",round((solution[defe])/(Final_C),3))
print("Error:",round(Err_def,3))

Defective Carbon 0.071
Error: 2.112


### Contribution of various C-X bonds for Aromatic and defective rings carbons (Theoretical value estimated through DFT)

### C-H 

In [240]:
# Function to count bonds between specified atoms
def count_bonds(atom1, atom2):
    selection_name = f"C-H"
    cmd.select(selection_name, f"({atom1}) within 1.8 of ({atom2})")
    bond_count = cmd.count_atoms(selection_name) - 1  # Subtract 1 to exclude the central atom
    return bond_count
    
carbon_atom = "elem C"
hydrogen_atom = "elem H"

carbon_hydrogen_bonds = count_bonds(carbon_atom, hydrogen_atom)
print(f"Number of C-H bonds: {carbon_hydrogen_bonds}")

Number of C-H bonds: 2817


In [241]:
# Take into account just bonds related to aromatic and defective carbons
print("C-H:",round((carbon_hydrogen_bonds-solution[methyl]*3-solution[ali_chain]*5-solution[ali_chain_2]*5-solution[carb]-solution[eth])/(solution[aro]+solution[defe]),3))

C-H: 0.139


### C-O-H

In [242]:
def count_three_atom_bonds(atom1, atom2, atom3):
    selection_name = f"C-O-H"
    cmd.select(selection_name, f"(({atom1}) within 1.8 of ({atom2})) or (({atom2}) within 1.8 of ({atom3})) or (({atom1}) within 1.8 of ({atom3}))")
    bond_count = cmd.count_atoms(selection_name) - 2  # Subtract 2 to exclude the central atoms
    
    return bond_count

carbon_atom = "elem C"
oxygen_atom = "elem O"
hydrogen_atom = "elem H"

carbon_oxygen_hydrogen_bonds = count_three_atom_bonds(carbon_atom, oxygen_atom, hydrogen_atom)
print(f"Number of C–O–H bonds: {carbon_oxygen_hydrogen_bonds}")

Number of C–O–H bonds: 4002


In [243]:
# Take into account just bonds related to aromatic and defective carbons
print("C-x-H:",round((carbon_oxygen_hydrogen_bonds-solution[carb]-solution[ester]*2-solution[eth])/(solution[aro]+solution[defe]),3))

C-x-H: 0.686


## Step 8: Organize the grid again and proceed to fix the helium density in LAMMPS

In [244]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [245]:
min_angle = 1
max_angle = 359.0

num_objects = len(object_names_new)

positions = []
used_positions = set()
angles = []
xmin, xmax = 0, round(system_size*0.007)
ymin, ymax = -round(system_size*0.007), 0
zmin, zmax = 0, round(system_size*0.007)

# Minimum distance between any two positions to avoid overlap
min_distance = round(system_size*0.001)
grid_divisions = int(math.ceil(num_objects ** (1 / 3)))

grid_size_x = min_distance
grid_size_y = min_distance
grid_size_z = min_distance

# Generate all possible grid positions
all_positions = [
    (xmin + i * grid_size_x, ymin + j * grid_size_y, zmin + k * grid_size_z)
    for i in range(grid_divisions)
    for j in range(grid_divisions)
    for k in range(grid_divisions)
]

random.shuffle(all_positions)
assert len(all_positions) >= num_objects, "Not enough positions for all objects."

# Assign positions to objects
for idx, name in enumerate(object_names_new):
    x, y, z = all_positions[idx]
    positions.append((x, y, z))

    rotation_angles = [random.uniform(min_angle, max_angle) for _ in range(3)]
    angles.append((rotation_angles, name))

    for axis, angle in zip(['x', 'y', 'z'], rotation_angles):
        rotation_command = "rotate {0}, {1}, {2}".format(axis, angle, name)
        cmd.do(rotation_command)

    # Translate the object to the new position
    cmd.translate((x, y, z), name)

cmd.refresh()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)

Number of objects: 56


In [246]:
# Obtain molecular weight distribution
object_masses = builder.Get_Object_Masses()
while not object_masses:
    object_masses = builder.Get_Object_Masses()

### Obtain molecular formula of the decorated clusters

In [247]:
def get_atom_counts(object_names_new):
    elements = ['C', 'H', 'N', 'O', 'S', 'Cl', 'Si']
    counts = [object_names_new]  # Start with the molecule name
    for element in elements:
        count = cmd.count_atoms(f'elem {element} and model {object_names_new}')
        counts.append(count)
    return counts

with open('Final_molecular_formula.txt', 'w') as file:
    file.write("Molecule,C,H,N,O,S,Cl,Si\n")
    
    for molecule in object_names_new:
        atom_counts = get_atom_counts(molecule)
        file.write(",".join(map(str, atom_counts)) + '\n')

print("Composition analysis complete. Results saved to 'Final_molecular_formula.txt'.")

Composition analysis complete. Results saved to 'Final_molecular_formula.txt'.


### Export the molecules as PDB file

In [248]:
# Select all atoms
cmd.select('all_atoms', 'all')

# Export the selection as a multi-file PDB
n_states = cmd.count_states('all_atoms')
pdb_files = []
for state in range(1, n_states + 1):
    cmd.frame(state)
    pdb_file = f'exported_molecule_{state}.pdb'
    cmd.save(pdb_file, 'all_atoms')
    pdb_files.append(pdb_file)

# Merge the multi-file PDB into a single file
merged_file = 'merged_molecule.pdb'
with open(merged_file, 'w') as outfile:
    for pdb_file in pdb_files:
        with open(pdb_file, 'r') as infile:
            outfile.write(infile.read())

# Save the merged molecule
cmd.load(merged_file, 'merged_molecule')
cmd.save('final_molecule.pdb', 'merged_molecule')

# Clean up the intermediate files
for pdb_file in pdb_files:
    os.remove(pdb_file)
os.remove(merged_file)

In [249]:
# Call the PyMOL aliases
cmd.do('colour')
cmd.do('orient')
# Change colors ans appereance
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

## Step 9: Convert the PDB file to Lammps data using OVITO

#### To define the x, y, z values use the following code:

In [250]:
HeliumDensity = float(input("Enter the value for Helium Density: "))  # kg/m3

Enter the value for Helium Density:  1.44


In [251]:
Temperature=298                             # K
Pressure=1                                  # atm
Density=(HeliumDensity*1000)/(10**(27))     # g/Å3
NA=6.022*10**(23)                           # molecules per mol
BoxVolume=1/((Density/newMW2)*NA)           # Å3
BoxLength=(BoxVolume)**(1/3)                # Å
print(" x, y, z:",round(BoxLength,3))


 x, y, z: 49.637


#### Open the PDB file in VMD, and in the console put the following commands:

cd Desktop

pbc set {x y z} -all


pbc box -center all

topo writelammpsdata biochar.data charge

# You can also convert it directly using OVITO, just open the file and export it as LAMMPS data using charge